In [1]:
import cv2
import numpy as np

In [2]:
yolo = cv2.dnn.readNet('yolov3.weights' ,'yolov3.cfg')

In [3]:
classes = []
with open("coco.names",'r') as fp:
    classes = fp.read().splitlines()

In [4]:
COLORS = np.random.uniform(0, 255, size=(len(classes),3) )

In [5]:
img = cv2.imread("HumAni.jpg")
imgH,imgW = img.shape[:2]

In [6]:
scale = 1/255
size = (320,320)

blob = cv2.dnn.blobFromImage(img ,scale ,size ,(0,0,0) ,swapRB=True ,crop=False)

In [7]:
yolo.setInput(blob)

In [8]:
output_layers_name = yolo.getUnconnectedOutLayersNames()

In [9]:
layer_output = yolo.forward(output_layers_name)

In [10]:
boxes = []
confidences = []
class_ids = []

for output in layer_output :
    for detection in output :
        score = detection[5:]
        class_id = np.argmax(score)
        confidence = score[class_id]
        
        if confidence>=0.7 : 
            centerX = int(detection[0]*imgW) 
            centerY = int(detection[1]*imgH) 
            w       = int(detection[2]*imgW) 
            h       = int(detection[3]*imgH)
            
            x       = int(centerX-w/2) 
            y       = int(centerY-h/2) 
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [11]:
indexes = cv2.dnn.NMSBoxes(
                            boxes                 ,
                            confidences           ,
                            score_threshold = 0.5 ,
                            nms_threshold   = 0.4
                          )

In [12]:
for i in indexes.flatten():
    x,y,w,h = boxes[i]
    
    obj = class_ids[i]
    
    label = str( classes[obj])
    color = COLORS[obj]
    
    confi = str( round(confidences[i] ,2))
    
    
    cv2.rectangle(img ,(x,y) ,(x+w,y+h) ,color ,2)
    cv2.putText(img ,label+','+confi ,(x-10,y-10) ,cv2.FONT_HERSHEY_SIMPLEX ,1,color ,1)

cv2.imshow("Object Detection" ,img)
cv2.waitKey(0)

-1